In [1]:
#基隆市
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time
# ---------- 等待頁面加載完成
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
# ----------

In [2]:
ISBN = "9789861206066"  # 三體，會有兩筆搜尋結果
# ISBN = "9789869109321"  # 二十一世紀資本論，會有一筆搜尋結果

In [3]:
# 設定 driver
my_options = Options()
my_options.add_argument("--incognito")  # 開啟無痕模式
# my_options.add_argument("--headless")  # 不開啟實體瀏覽器

capabilities = DesiredCapabilities.CHROME
capabilities["pageLoadStrategy"] = "none"

In [4]:
# 初始化 df_DataFrame
df_result = []

# 開啟 Chrome，設定等待元素最多 10 秒
driver = webdriver.Chrome(options=my_options, desired_capabilities=capabilities)
wait = WebDriverWait(driver, 10)

In [5]:
ISBN = "9789861206066"  # 三體，會有兩筆搜尋結果
# ISBN = "9789573317241"  # 哈利波特：神秘的魔法石，會有一筆搜尋結果，但多頁
# def webpac_cfm_crawler()
url = f"https://webpac.klccab.gov.tw/webpac/search.cfm?m=as&k0={ISBN}&t0=i&c0=and&y10=&y20=&cat0=&dt0=&l0=&lv0=&lc0="


In [ ]:
driver.get(url)

time.sleep(5)  # 基隆的系統太詭異了，強制等待
soup = BeautifulSoup(driver.page_source, "html.parser")
results = len(soup.find_all("div", "list_box"))
if results < 2:
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.list.list_border")))
    tgt = pd.read_html(driver.page_source)[0]
else:
    tgt = []
    for li in soup.find_all("div", "list_box"):
        url_temp = "https://webpac.klccab.gov.tw/webpac/" + li.find("a", "btn")["href"]
        driver.get(url_temp)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.list.list_border")))
        tgt.append(pd.read_html(driver.page_source, encoding="utf-8")[0])
    tgt = pd.concat(tgt, axis=0, ignore_index=True)